# Libraries

In [91]:
import networkx as nx
import pygame
import time
import random
import numpy
import matplotlib.pyplot as plt

# Global Variables

In [92]:
# GLOBAL VARIABLES
WHITE = (255,255,255)
BLACK = (0,0,0)
GREY = (194,194,194)
BLUE = (25, 166, 194)
DARK_BLUE = (21, 134, 157)
RED = (255, 0, 0)
ORANGE = (234, 108, 94)
DARK_ORANGE = (236, 44, 22)
GREEN = (0, 255, 0)
LIME = (66, 213, 30)
DARK_LIME = (48, 157, 21)
BROWN = (205, 118, 66)
DARK_BROWN = (135, 76, 41)
SHUFFLE_SIZE = 10000
OFFSET = 100


# Functions

In [93]:
def round_better(number, significant_figures):
    from math import log10, floor
    
    if number == 0:
        return 0
    else:
        decimal_places = significant_figures - int(floor(log10(abs(number)))) - 1
        rounded_number = round(number, decimal_places)
        return rounded_number
    
def draw_contour(screen, size, color):
    pygame.draw.rect(screen, color, pygame.Rect(400-(size/2) + OFFSET, 300-(size/2), size, size))
    
def draw_rounded_rect(screen, color, x, y, w, h, radius):
    pygame.draw.rect(screen, color, (x + radius, y, w - 2 * radius, h))
    pygame.draw.rect(screen, color, (x, y + radius, w, h - 2 * radius))
    pygame.draw.circle(screen, color, (x + radius, y + radius), radius)
    pygame.draw.circle(screen, color, (x + w - radius, y + radius), radius)
    pygame.draw.circle(screen, color, (x + radius, y + h - radius), radius)
    pygame.draw.circle(screen, color, (x + w - radius, y + h - radius), radius)
    
def draw_text(screen, text, x, y, font_size, color):
    font = pygame.font.Font(None, font_size)
    text = font.render(text, True, color)
    screen.blit(text, (x, y))
    
def draw_pieces(screen,positions):
    row = -1
    for i in range(9):
        if (i % 3 == 0): row += 1 
        if (positions[i] != ' '):
            draw_rounded_rect(screen, BLACK, (1.485*(i % 3) + 1.0017)*124 + OFFSET, (7.67*(row % 3) + 1.0017)*24, 184, 184, 10)
            draw_rounded_rect(screen, WHITE, (1.485*(i % 3) + 1.0017)*(124) + 2 + OFFSET, (7.67*(row % 3) + 1.0017)*(24) + 2, 180, 180, 10) 
            draw_text(screen, str(positions[i]), (1.485*(i % 3) + 1.53)*124 + OFFSET,(7.67*(row % 3) + 3)*24,150,BLACK)

def draw_path(screen, path, color1, color2, text, delay):
    for n in path:
        screen.fill(WHITE)
        draw_contour(screen, 600, color1)
        draw_contour(screen, 585, color2)
        draw_contour(screen, 552, GREY)
        draw_pieces(screen, n)
        draw_text(screen, text, 25, 280, 50, color2)
        pygame.display.flip()
        pygame.time.delay(delay)

def draw_time(screen, elapsed_time):
    draw_text(screen, "TIME:", 25, 100, 50, ORANGE)
    draw_text(screen, str(round_better(elapsed_time,2)) + " s", 25, 135, 45, ORANGE)
    pygame.display.flip()
    pygame.time.delay(4000)

def draw_vs(screen, elapsed_time1, elapsed_time2, path1, path2):
    draw_text(screen, "A*:", 25, 100, 50, BLUE)
    draw_text(screen, str(elapsed_time1) + "s", 25, 135, 50, BLUE)
    draw_text(screen, "PATH: " + str(len(path1)), 25, 170, 50, BLUE)

    draw_text(screen, "MM:", 25, 250, 50, ORANGE)
    draw_text(screen, str(elapsed_time2) + "s", 25, 285, 50, ORANGE)
    draw_text(screen, "PATH: " + str(len(path2)), 25, 320, 50, ORANGE)
    pygame.display.flip()
    pygame.time.delay(4000)

def draw_path_size(screen, path):
    draw_text(screen, "PATH SIZE:", 25, 100, 45, ORANGE)
    draw_text(screen, str(len(path)), 25, 135, 45, ORANGE)
    pygame.display.flip()
    pygame.time.delay(3000)

def draw_bar(screen, size, y, color):
    size = size * 60
    x = (800 - size) // 2
    pygame.draw.rect(screen, BLACK, (x + 130, y, size, 35))
    pygame.draw.rect(screen, color, (x + 130 + 2, y + 2, size - 4, 35 - 4))

def draw_white(screen):
    pygame.draw.rect(screen, WHITE, (250, 0, 600, 600))

def draw_pancake(screen, secuence):
    for i in range(len(secuence)):
        draw_bar(screen, secuence[i], 35 * i + 200, BLUE)

def draw_pancake_split(screen, secuence, cut):
    for i in range(0, cut):
        draw_bar(screen, secuence[i], 35 * i + 150, ORANGE)
    
    for i in range(cut, len(secuence)):
        draw_bar(screen, secuence[i], 35 * i + 200, BLUE)

def flip_secuence(secuence, cut):
    secuence = secuence[:0] + secuence[0:cut][::-1] + secuence[cut:]
    return secuence

def clear_screen(screen):
    pygame.draw.rect(screen, WHITE, (0, 0, 800,600))

def draw_pancake_secuence(screen, path):
    draw_text(screen, "Solving...", 25, 280, 50, BLUE)
    for i in range(len(path) - 1):
        path_secuence = [int(i) for i in path[i][0]]
        cut = int(path[i][1])

        draw_pancake(screen, path_secuence)
        pygame.display.flip()
        pygame.time.delay(250)
        draw_white(screen)

        draw_pancake_split(screen, path_secuence, cut)
        pygame.display.flip()
        pygame.time.delay(250)
        draw_white(screen)

        path_secuence = flip_secuence(path_secuence, cut)

        draw_pancake_split(screen, path_secuence, cut)
        pygame.display.flip()
        pygame.time.delay(250)
        draw_white(screen)

        draw_pancake(screen, path_secuence)
        pygame.display.flip()
        pygame.time.delay(250)
        draw_white(screen)
    clear_screen(screen)

def draw_performance(screen, elapsed_time, path):
    draw_text(screen, "TIME:", 25, 100, 45, ORANGE)
    draw_text(screen, str(round_better(elapsed_time,2)) + " s", 25, 135, 45, ORANGE)
    draw_text(screen, "PATH SIZE:", 25, 190, 45, DARK_BLUE)
    draw_text(screen, str(len(path)), 25, 225, 45, DARK_BLUE)
    draw_pancake(screen, path)
    draw_text(screen, "Solved", 25, 280, 50, LIME)
    pygame.display.flip()

# PyGame

In [94]:
class Button:
    def __init__(self, x, y, width, height, text, color, hover_color):
        self.rect = pygame.Rect(x, y, width, height)
        self.text = text
        self.color = color
        self.hover_color = hover_color
        self.font = pygame.font.Font(None, 36)
    
    def draw(self, screen):
        mouse_pos = pygame.mouse.get_pos()
        if self.rect.collidepoint(mouse_pos):
            pygame.draw.rect(screen, self.hover_color, self.rect)
        else:
            pygame.draw.rect(screen, self.color, self.rect)
        
        text_surf = self.font.render(self.text, True, BLACK)
        text_rect = text_surf.get_rect(center=self.rect.center)
        screen.blit(text_surf, text_rect)
    
    def is_clicked(self, event):
        if event.type == pygame.MOUSEBUTTONDOWN:
            if self.rect.collidepoint(event.pos):
                return True
        return False

    def set_text(self, text):
        self.text = text

    def set_color(self, color):
        self.color = color

# 8Puzzle

## Search Algorithms

### AStar8Puzzle Class

In [95]:
class AStar8Puzzle:
    @staticmethod
    def find_blank(array_matriz):
        index= array_matriz.index(' ')
        return index
    
    @staticmethod
    def get_score_index(index_state, index_solucion):
        row_1 = [0, 1, 2]
        row_2 = [3, 4, 5]
        row_3 = [6, 7, 8]

        col_1 = [0, 3, 6]
        col_2 = [1, 4, 7]
        col_3 = [2, 5, 8]

        state_x = 0
        state_y = 0
        sol_x = 0
        sol_y = 0

        if index_solucion in row_1:
            sol_y = 0
        if index_solucion in row_2:
            sol_y = 1
        if index_solucion in row_3:
            sol_y = 2
        if index_state in row_1:
            state_y = 0
        if index_state in row_2:
            state_y = 1
        if index_state in row_3:
            state_y = 2
        if index_solucion in col_1:
            sol_x = 0
        if index_solucion in col_2:
            sol_x = 1
        if index_solucion in col_3:
            sol_x = 2
        if index_state in col_1:
            state_x = 0
        if index_state in col_2:
            state_x = 1
        if index_state in col_3:
            state_x = 2

        return abs(sol_x - state_x) + abs(sol_y - state_y)
    
    @staticmethod
    def get_score(lista_1nodo, lista_goal):
        score=0
        for i in range(len(lista_1nodo)):
            if i==0:
                nodo_index=lista_1nodo.index(' ')
                goal_index=lista_goal.index(' ')
            else:
                nodo_index=lista_1nodo.index(i)
                goal_index=lista_goal.index(i)
            score= score + AStar8Puzzle.get_score_index(nodo_index,goal_index)
            
        return score
    
    @staticmethod
    def get_n_sucesores(array_matriz):
        acciones = []
        index = AStar8Puzzle.find_blank(array_matriz)
        if index  in [0,1,2,3,4,5]:
            acciones.append('down')
        if index in [0,3,6,1,4,7]:
            acciones.append('right')
        if index in [2,5,8,1,4,7]:
            acciones.append('left')
        if index in [6,7,8,3,4,5]:
            acciones.append('up')

        return acciones
    
    @staticmethod
    def generate_n_sucesores(lista_acciones, nodo_anterior):
        index=AStar8Puzzle.find_blank(nodo_anterior)
        lista_n_sucesores=[]
        while lista_acciones:
            aux=-1
            lista_p=nodo_anterior.copy()
            accion=lista_acciones.pop()
            if accion=="up":
                aux=lista_p[index]
                lista_p[index]=lista_p[index-3]
                lista_p[index-3]=aux
            if accion=="down":
                aux=lista_p[index]
                lista_p[index]=lista_p[index+3]
                lista_p[index+3]=aux
            if accion== "right":
                aux=lista_p[index]
                lista_p[index]=lista_p[index+1]
                lista_p[index+1]=aux
            if accion=="left":
                aux=lista_p[index]
                lista_p[index]=lista_p[index-1]
                lista_p[index-1]=aux

            lista_n_sucesores.append(lista_p)

        return lista_n_sucesores
    
    @staticmethod
    def peso_heuristica(tupla_de_lista_tuplas):
        return tupla_de_lista_tuplas[2]+len(tupla_de_lista_tuplas[1])
    
    @staticmethod
    def get_min_heuristic(lista_nodos_posible):
        nodo_mejor_heuristica=min(lista_nodos_posible, key=AStar8Puzzle.peso_heuristica)
        return nodo_mejor_heuristica
    
    @staticmethod
    def formato_comp_tuplas(nodo_actual, path_p, goal_node, reached):
        acciones_posibles=AStar8Puzzle.get_n_sucesores(nodo_actual)
        lista_sucesores=AStar8Puzzle.generate_n_sucesores(acciones_posibles, nodo_actual)
        sucesores_tuplas=[]
        for node in lista_sucesores:
            node_tupla=tuple(node)

            if node_tupla not in reached:
                path_nuevo=path_p+[node]
                score_nueva= AStar8Puzzle.get_score(node,goal_node)
                sucesores_tuplas.append((node,path_nuevo,score_nueva))
                reached.add(node_tupla)

        return sucesores_tuplas
    
    @staticmethod
    def imprimir_matriz(array_m):
        print("Matriz :")
        for i in range(0, len(array_m), 3):
            print(' '.join(map(str, array_m[i:i+3])))
        ####print("Fin de Matriz")
        print('-' * 20)
              
    @staticmethod
    def solve(scramble, solution):
        start_time = time.time()
        
        full_path = []
        reached=set()
        fila_a_star=[(scramble,[scramble],AStar8Puzzle.get_score(scramble,solution))]
        i=0
        while fila_a_star:
            i=i+1
            zip_nodo=AStar8Puzzle.get_min_heuristic(fila_a_star)

            fila_a_star.remove(zip_nodo)
            nodo_p, path_p, score_p = zip_nodo
            full_path.append(nodo_p)

            if nodo_p==solution:
                end_time = time.time()
                elapsed_time = end_time - start_time
                return nodo_p, path_p,full_path, score_p, elapsed_time

            tuplas_sucesores=AStar8Puzzle.formato_comp_tuplas(nodo_p,path_p,solution,reached)
            fila_a_star= fila_a_star+tuplas_sucesores
    

### Shuffle8Puzzle Class

In [96]:
class Shuffle8Puzzle:
    @staticmethod
    def pop_aleatorio(nodos):
        if nodos:
            index= random.randrange(len(nodos))
            return nodos.pop(index)
        else:
            print ('Error de índice en pop_aleatorio')
            
    @staticmethod
    def generate_nodo_inicial(nodo_solucion, complejidad):
        fila_random=[nodo_solucion]
        reached=set()
        for i in range(complejidad):
        
            nodo=Shuffle8Puzzle.pop_aleatorio(fila_random)
            tupla_nodo=tuple(nodo)
            
            if i+1==complejidad:
                return nodo
            
            if tupla_nodo not in reached:
                reached.add(tupla_nodo)
                acciones=AStar8Puzzle.get_n_sucesores(nodo)
                random.shuffle(acciones)
                
                nodos_sucesores=AStar8Puzzle.generate_n_sucesores(acciones, nodo)
                
                fila_random=fila_random+nodos_sucesores

    @staticmethod
    def contar_inversiones(lista):
        flat_list = [num for num in lista if num != ' ']
        inversiones = 0
        for i in range(len(flat_list)):
            for j in range(i + 1, len(flat_list)):
                if flat_list[i] > flat_list[j]:
                    inversiones += 1
        return inversiones

### MM8PuzzleClass

In [97]:
class MM8Puzzle:
    @staticmethod
    def reconstruct_path(parentF, parentB, meeting_node):
        # Path from start to meeting node
        pathF = []
        node = meeting_node
        while node is not None:
            pathF.append(node)
            node = parentF[node]
        pathF.reverse()

        # Path from meeting node to goal
        pathB = []
        node = parentB[meeting_node]
        while node is not None:
            pathB.append(node)
            node = parentB[node]

        pathFinal = pathF + pathB

        return pathFinal

    @staticmethod
    def getpr(node, g, f):
        return max(f[node], 2*g[node])
    
    @staticmethod
    def getprmin(g, f, Open):
        minTemp = {}
        for key  in g:
            if key in Open:
                minTemp[key] = MM8Puzzle.getpr(key, g, f)
        minimumKey = min(minTemp, key=minTemp.get)
        return [minimumKey, minTemp[minimumKey]]

    @staticmethod
    def solve(start_node, goal_node):
        start_time = time.time()
        full_path = []
        # Listas con los valores g y f; g es la distancia de su nodo origen a el, f es esta distancia mas la heuristica
        gF = {tuple(start_node): 0}
        gB = {tuple(goal_node): 0}
        fF = {tuple(start_node): AStar8Puzzle.get_score(start_node, goal_node)}
        fB = {tuple(goal_node): AStar8Puzzle.get_score(goal_node, start_node)}
        # Open y Close para ambos lados
        OpenF = {tuple(start_node): (start_node,[start_node],AStar8Puzzle.get_score(start_node,goal_node))}
        OpenB = {tuple(goal_node): (goal_node,[goal_node],AStar8Puzzle.get_score(goal_node,start_node))}
        ClosedF = set()
        ClosedB = set()
        #reached.add(node_tupla)
        # U se encarga de saber si encontramos un camino y si es el optimo
        U = float('inf')
        # Parent list for the path
        parentF = {tuple(start_node): None}
        parentB = {tuple(goal_node): None}
        meeting_node = None


        # Loop de algorito
        while OpenF and OpenB:
            # C determina la direccion que debe tomar el algoritmo; forward o backward
            C = min(MM8Puzzle.getprmin(gF, fF, OpenF)[1], MM8Puzzle.getprmin(gB, fB, OpenB)[1])
            #Se checa si ya llegamos al camino optimo
            if U <= max(C, min(fF.values()), min(fB.values()), min(gF.values()) + min(gB.values()) + 1):
                end_time = time.time()
                execution_time = end_time - start_time
                return MM8Puzzle.reconstruct_path(parentF, parentB, meeting_node), full_path, execution_time
            # If para Backward o Forward
            if C == MM8Puzzle.getprmin(gF, fF, OpenF)[1]:
                #n = min(OpenF, key=lambda x: (gF[x], -graph.nodes[x]['priority']))
                # Nodo con el mejor costo actual??
                n = MM8Puzzle.getprmin(gF, fF, OpenF)[0]
                full_path.append(n)


                ClosedF.add(n)

                tuplaSucesores = AStar8Puzzle.formato_comp_tuplas(list(n), OpenF[n][1], goal_node, ClosedF)
                del OpenF[n]

                listaSucesores = {}
                for child in tuplaSucesores:
                    listaSucesores[tuple(child[0])] = child
                #Manejamos los child nodes
                for c in listaSucesores:
                    if (c in OpenF or c in ClosedF) and gF.get(c, float('inf')) <= gF[n] + 1:
                        continue

                    if (c in OpenF or c in ClosedF):
                        if c in OpenF: del OpenF[c]
                        if c in ClosedF: ClosedF.remove(c)
                    gF[c] = gF[n] + 1
                    fF[c] = gF[c] + AStar8Puzzle.get_score(c, goal_node)

                    OpenF[c] = (list(c),listaSucesores[c][1],listaSucesores[c][2])
                    parentF[c] = n
                    if c in OpenB:
                        U = min(U, gF[c] + gB[c])
                        meeting_node = c
            else: # Hacemos lo mismo de arriba pero para el backward search
                #n = min(OpenB, key=lambda x: (gB[x], -graph.nodes[x]['priority']))
                n = MM8Puzzle.getprmin(gB, fB, OpenB)[0]
                ClosedB.add(n)

                tuplaSucesores = AStar8Puzzle.formato_comp_tuplas(list(n), OpenB[n][1], start_node, ClosedB)
                del OpenB[n]

                listaSucesores = {}
                for child in tuplaSucesores:
                    listaSucesores[tuple(child[0])] = child

                for c in listaSucesores:
                    if (c in OpenB or c in ClosedB) and gB.get(c, float('inf')) <= gB[n] + 1:
                        continue
                    if (c in OpenB or c in ClosedB):
                        if c in OpenB: del OpenB[c]
                        if c in ClosedB: ClosedB.remove(c)
                    gB[c] = gB[n] + 1
                    fB[c] = gB[c] + AStar8Puzzle.get_score(c, start_node)

                    OpenB[c] = (list(c),listaSucesores[c][1],listaSucesores[c][2])
                    parentB[c] = n
                    if c in OpenF:
                        U = min(U, gF[c] + gB[c])
                        meeting_node = c
        return float('inf')

## Main Game Function

In [98]:
def run_8Puzzle():
    pygame.init()

    screen_width = 800
    screen_height = 600
    screen = pygame.display.set_mode((screen_width, screen_height))

    pygame.display.set_caption("8-Puzzle")
    clock = pygame.time.Clock()
    fps = 30

    scramble_button = Button(25, 250, 150, 100, "Scramble", DARK_ORANGE, ORANGE)
    a_star_button = Button(25, 100, 150, 100, "A*", DARK_BLUE, BLUE)
    mm_button = Button(25, 400, 150, 100, "MM", DARK_LIME, LIME)
    vs_button = Button(25, 250, 150, 100, "VS", DARK_BROWN, BROWN)

    solution = [1, 2, 3, 4, 5, 6, 7, 8, ' ']
    state = solution
    button_state = "Scramble"  # Initial button state

    running = True

    while running:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False

            if button_state == "Scramble" and scramble_button.is_clicked(event):
                state = Shuffle8Puzzle.generate_nodo_inicial(solution, SHUFFLE_SIZE)
                button_state = "Choose Algorithm"
            elif button_state == "Choose Algorithm":
                if a_star_button.is_clicked(event):
                    nodo_p, path_p, full_path, score_p, elapsed_time = AStar8Puzzle.solve(state, solution)
                    draw_path(screen, full_path, DARK_BLUE, BLUE, "Solving...",int(30*(200/SHUFFLE_SIZE)))
                    draw_time(screen, elapsed_time)
                    draw_path(screen, path_p, DARK_LIME, LIME, "Solution",200)
                    draw_path_size(screen, path_p)

                    state = solution
                    button_state = "Scramble"
                    scramble_button.set_text("Scramble")
                    scramble_button.set_color(DARK_ORANGE)

                elif mm_button.is_clicked(event):
                    path_p, full_path, elapsed_time = MM8Puzzle.solve(state, solution)
                    draw_path(screen, full_path, DARK_BLUE, BLUE, "Solving...",int(30*(200/SHUFFLE_SIZE)))
                    draw_time(screen, elapsed_time)
                    draw_path(screen, path_p, DARK_LIME, LIME, "Solution",200)
                    draw_path_size(screen, path_p)

                    state = solution
                    button_state = "Scramble"
                    scramble_button.set_text("Scramble")
                    scramble_button.set_color(DARK_ORANGE)

                elif vs_button.is_clicked(event):
                    state1 = state
                    nodo_p, path_p1, full_path, score_p, elapsed_time1 = AStar8Puzzle.solve(state1, solution)
                    draw_path(screen, full_path, DARK_BLUE, BLUE, "Solving...",int(30*(200/SHUFFLE_SIZE)))
                    draw_time(screen, elapsed_time1)
                    draw_path(screen, path_p1, DARK_LIME, LIME, "Solution",200)
                    draw_path_size(screen, path_p1)

                    state1 = solution
                    button_state = "Scramble"
                    scramble_button.set_text("Scramble")
                    scramble_button.set_color(DARK_ORANGE)

                    path_p2, full_path, elapsed_time2 = MM8Puzzle.solve(state, solution)
                    draw_path(screen, full_path, DARK_BLUE, BLUE, "Solving...",int(30*(200/SHUFFLE_SIZE)))
                    draw_time(screen, elapsed_time2)
                    draw_path(screen, path_p2, DARK_LIME, LIME, "Solution",200)
                    draw_path_size(screen, path_p2)

                    clear_screen(screen)
                    draw_vs(screen, elapsed_time1, elapsed_time2, path_p1, path_p2)
                    state = solution
                    button_state = "Scramble"
                    scramble_button.set_text("Scramble")
                    scramble_button.set_color(DARK_ORANGE)



        # Fill the screen with white color
        screen.fill(WHITE)

        # Draw the initial state of the puzzle
        draw_contour(screen, 600, DARK_BROWN)
        draw_contour(screen, 585, BROWN)
        draw_contour(screen, 552, GREY)
        draw_pieces(screen, state)

        if button_state == "Scramble":
            scramble_button.draw(screen)
        elif button_state == "Choose Algorithm":
            a_star_button.draw(screen)
            mm_button.draw(screen)
            vs_button.draw(screen)

        # Update the display
        pygame.display.flip()

        # Limit the frame rate
        clock.tick(fps)

    # Quit Pygame
    pygame.quit()

# Pancake

## Pancake Class

In [99]:
class Pancake:
    def __init__(self, size=8, preset=None):
        # Makes our pancake or uses the one provided through preset
        if preset == None:
            self.pancake = []
            self.size = size
            for i in range(0, size):
                self.pancake.append(i+1)
            random.shuffle(self.pancake)
        else:
            self.pancake = preset
            self.size = len(preset)

        # We make the solution pancake
        self.solve = []
        for i in range(0, self.size):
            self.solve.append(i+1)

        # Add both to the graph, the name of the node will be a string with the numbers of the pancake
        self.graph = nx.Graph()
        self.graph.add_node(self.makePancakeintoStr(self.pancake), pancake=self.pancake)
        self.graph.add_node(self.makePancakeintoStr(self.solve), pancake=self.solve)

        # TO BE REMOVED
        self.counters = [0,1000000]
    
    # This will take the pancake in its normal form (a list) and return it as a string, mostly used for naming
    def makePancakeintoStr(self, pancake):
        name = ''
        for ele in pancake:
                name = name + str(ele)
        return name
    def makeStrintoPancake(self, stringP):
        panc = []
        for ele in stringP:
                panc.append(int(ele))
        return panc
    
    # This will flip the pancakes and return a list with the new positions, Pos is the position from top to bottom at which we'll flip
    def getFlip(self, pos, pancake):
        if (pos < 1 or pos > self.size):
            return "Invald Pos"

        flip = pancake[0:pos]
        flip.reverse()

        new = flip + pancake[pos:]
        return new
    
    # We will get all possible flips of a certaing pancake and return them as a list of lists(pancakes)
    def getSuccesors(self, pancake=None):
        if pancake == None: pancake = self.pancake
        
        listofSuccesors = []
        for i in range(2, self.size + 1):
            listofSuccesors.append(self.getFlip(i, pancake))

        return listofSuccesors

    # This will use the possibilities to make them into nodes and add them to the graph (THIS IS THE PREVIOUS VERSION)
    def getSuccesorNodesPREV(self, node, counter):
        possibilities = self.getSuccesors(self.graph.nodes[node]["pancake"])
        numC = 0
        for i in possibilities:
            numC += 1
            if (counter == 0):
                self.graph.add_node(str(self.counters[0]), pancake=i)
                self.graph.add_edge(node, str(self.counters[0]))
                self.counters[0] += 1
            else:
                self.graph.add_node(str(self.counters[1]), pancake=i)
                self.graph.add_edge(node, str(self.counters[1]))
                self.counters[1] += 1

    # This will use the possibilities to make them into nodes and add them to the graph 
    def getSuccesorNodes(self, node):
        possibilities = self.getSuccesors(self.graph.nodes[node]["pancake"])
        numC = 1
        for i in possibilities:
            numC += 1
            name = self.makePancakeintoStr(i)
            if (self.graph.has_node(name)):
                self.graph.add_edge(node, name)
            else:
                self.graph.add_node(name, pancake=i)
                self.graph.add_edge(node, name)

    # This uses the GAP method to get the euristic of a certain pancake step
    def heuristicGAP(self, pancake):
        n = len(pancake)
        gaps = 0

        # Check gaps between adjacent pancake
        for i in range(n - 1):
            if pancake[i] > pancake[i + 1]:
                gaps += 1

        # Check boundary gaps
        if pancake[0] != 1:
            gaps += 1  # Top boundary gap
        if pancake[-1] != n:
            gaps += 1  # Bottom boundary gap

        return gaps

    # This maps a specific state to a a solved state and thus is used to get the Backwards heuristic
    def heuristicGAPBackwards(self, pancakes, target):
        n = len(pancakes)
        gaps = 0

        # Create a mapping from pancake value to its position in the target configuration
        target_pos = {val: idx for idx, val in enumerate(target)}

        # Check gaps between adjacent pancakes
        for i in range(n - 1):
            if target_pos[pancakes[i]] > target_pos[pancakes[i + 1]]:
                gaps += 1

        # Check boundary gaps based on the target configuration
        if pancakes[0] != target[0]:
            gaps += 1  # Top boundary gap
        if pancakes[-1] != target[-1]:
            gaps += 1  # Bottom boundary gap

        return gaps
    
    
    # Funcion de priority Forward
    def getprF(self, node, gF, fF):
        return max(fF[node], 2*gF[node])

    # Funcion de priority Backward
    def getprB(self, node, gB, fB):
        return max(fB[node], 2*gB[node])

    # Funcion para sacar la priority menos costosa en Forward
    def getprminF(self, gF, fF, OpenF):
        minTemp = {}
        for key  in gF:
            if key in OpenF:
                minTemp[key] = self.getprF(key, gF, fF)
        minimumKey = min(minTemp, key=minTemp.get)

        return [minimumKey, minTemp[minimumKey]]

    # Funcion para sacar la priority menos costosa en Backward
    def getprminB(self, gB, fB, OpenB):
        minTemp = {}
        for key  in gB:
            if key in OpenB:
                minTemp[key] = self.getprB(key, gB, fB)
        minimumKey = min(minTemp, key=minTemp.get)

        return [minimumKey, minTemp[minimumKey]]

    # This uses the parents to determine the final path taken to solution
    def reconstruct_path(self, parentF, parentB, meeting_node):
        # Path from start to meeting node
        pathF = []
        node = meeting_node
        while node is not None:
            pathF.append([node, len(node)])
            node = parentF[node]
        pathF.reverse()

        # Path from meeting node to goal
        pathB = []
        node = parentB[meeting_node]
        while node is not None:
            pathB.append([node, len(node)])
            node = parentB[node]

        pathFinal = pathF + pathB
        for i in range(0, len(pathFinal) - 1):
                    for char in range(0, len(pathFinal[i][0])): 
                        if (pathFinal[i][0][char] == pathFinal[i + 1][0][char]) and (pathFinal[i][0][0] == pathFinal[i + 1][0][char - 1]):
                            pathFinal[i][1] = char 
                            break

        return pathFinal

    def a_star(self):
        start_time = time.time()

        start_node = self.makePancakeintoStr(self.pancake)
        goal_node = self.makePancakeintoStr(self.solve)

        g = {start_node: 0}
        f = {start_node: self.heuristicGAP(self.graph.nodes[start_node]["pancake"])}

        Open = {start_node: 0}
        Closed = {}

        parent = {start_node: None}

        while Open:
            ser = {}
            for key in f:
                if key in Open: ser[key] = f[key]
            n = min(ser, key=ser.get)
            if (n == goal_node):
                pathF = []
                node = goal_node
                while node is not None:
                    pathF.append([node, len(node)])
                    node = parent[node]
                pathF.reverse()

                for i in range(0, len(pathF) - 1):
                    for char in range(0, len(pathF[i][0])): #char in pathF[i][0]:
                        if (pathF[i][0][char] == pathF[i + 1][0][char]) and (pathF[i][0][0] == pathF[i + 1][0][char - 1]):
                            pathF[i][1] = char 
                            break
                
                end_time = time.time()
                execution_time = end_time - start_time
                return pathF, execution_time

            self.getSuccesorNodes(n)
            for c in self.graph.neighbors(n):
                curr_cost = g[n] + 1
                if c in Open:
                    if g[c] <= curr_cost: continue
                elif c in Closed:
                    if g[c] <= curr_cost: continue
                    Open[c] = Closed[c]
                    del Closed[c]
                else:
                    Open[c] = curr_cost
                g[c] = curr_cost
                f[c] = curr_cost + self.heuristicGAP(self.graph.nodes[c]['pancake'])
                parent[c] = n

            Closed[n] = Open[n]
            del Open[n]   
        return "error"             


    # Implementation of the MM algorithmn with pancake problem
    def mm_search(self, start_node, goal_node):
        start_time = time.time()

        # We determine the start and goal nodes
        start_node = self.makePancakeintoStr(self.pancake)
        goal_node = self.makePancakeintoStr(self.solve)

        # Listas con los valores g y f; g es la distancia de su nodo origen a el, f es esta distancia mas la heuristica
        gF = {start_node: 0}
        gB = {goal_node: 0}
        fF = {start_node: self.heuristicGAP(self.graph.nodes[start_node]["pancake"])}
        fB = {goal_node: self.heuristicGAPBackwards(self.graph.nodes[goal_node]["pancake"], self.solve)}

        # Open y Close para ambos lados
        OpenF = {start_node: 0}
        OpenB = {goal_node: 0}
        ClosedF = {}
        ClosedB = {}

        # U se encarga de saber si encontramos un camino y si es el optimo
        U = float('inf')

        # Parent list for the path
        parentF = {start_node: None}
        parentB = {goal_node: None}
        meeting_node = None

        # Loop de algorito
        while OpenF and OpenB:
            # C determina la direccion que debe tomar el algoritmo; forward o backward
            C = min(self.getprminF(gF, fF, OpenF)[1], self.getprminB(gB, fB, OpenB)[1])

            #Se checa si ya llegamos al camino optimo
            if U <= max(C, min(fF.values()), min(fB.values()), min(gF.values()) + min(gB.values()) + 1):
                #pos = nx.spring_layout(self.graph)
                #nx.draw(self.graph, pos, with_labels=True, node_size=200, node_color="skyblue", font_size=6, font_weight="bold")
                #labels = nx.get_edge_attributes(self.graph, "flip")
                #nx.draw_networkx_edge_labels(self.graph, pos, edge_labels=labels)
                #plt.show()

                end_time = time.time()
                execution_time = end_time - start_time
                return self.reconstruct_path(parentF, parentB, meeting_node), execution_time

            # If para Backward o Forward
            if C == self.getprminF(gF, fF, OpenF)[1]:
                #n = min(OpenF, key=lambda x: (gF[x], -graph.nodes[x]['priority']))
                # Nodo con el mejor costo actual??
                n = self.getprminF(gF, fF, OpenF)[0]
                
                ClosedF[n] = OpenF[n]
                del OpenF[n]

                self.getSuccesorNodes(n)
                #Manejamos los child nodes
                for c in self.graph.neighbors(n): 
                    if c in dict(OpenF, **ClosedF) and gF.get(c, float('inf')) <= gF[n] + 1:
                        continue
                    
                    if c in dict(OpenF, **ClosedF):
                        if c in OpenF: del OpenF[c]
                        if c in ClosedF: del ClosedF[c]
                    gF[c] = gF[n] + 1
                    fF[c] = gF[c] + self.heuristicGAP(self.graph.nodes[c]["pancake"])
                    
                    OpenF[c] = gF[c]
                    parentF[c] = n

                    if c in OpenB:
                        U = min(U, gF[c] + gB[c])
                        meeting_node = c
            else: # Hacemos lo mismo de arriba pero para el backward search
                #n = min(OpenB, key=lambda x: (gB[x], -graph.nodes[x]['priority']))
                n = self.getprminB(gB, fB, OpenB)[0]

                ClosedB[n] = OpenB[n]
                del OpenB[n]

                self.getSuccesorNodes(n)
                for c in self.graph.neighbors(n):
                    if c in dict(OpenB, **ClosedB) and gB.get(c, float('inf')) <= gB[n] + 1:
                        continue

                    if c in dict(OpenB, **ClosedB):
                        if c in OpenB: del OpenB[c]
                        if c in ClosedB: del ClosedB[c]
                    gB[c] = gB[n] + 1
                    fB[c] = gB[c] + self.heuristicGAPBackwards(self.graph.nodes[c]["pancake"], self.solve)
                    
                    OpenB[c] = gB[c]
                    parentB[c] = n

                    if c in OpenF:
                        U = min(U, gF[c] + gB[c])
                        meeting_node = c
        return float('inf')

## Main Game Function

In [100]:
def run_Pancake():
    pygame.init()

    screen_width = 800
    screen_height = 600
    screen = pygame.display.set_mode((screen_width, screen_height))

    pygame.display.set_caption("Pancake")
    clock = pygame.time.Clock()
    fps = 30

    scramble_button = Button(25, 250, 150, 100, "Scramble", DARK_ORANGE, ORANGE)
    a_star_button = Button(25, 100, 150, 100, "A*", DARK_BLUE, BLUE)
    mm_button = Button(25, 400, 150, 100, "MM", DARK_LIME, LIME)
    vs_button = Button(25, 250, 150, 100, "VS", DARK_BROWN, BROWN)

    button_state = "Scramble"  # Initial button state
    solved = [1,2,3,4,5,6,7,8]
    state = solved
    running = True

    while running:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False

            if button_state == "Scramble" and scramble_button.is_clicked(event):
                pancake = Pancake()
                state = pancake.pancake
                pygame.display.flip()
                button_state = "Choose Algorithm"

            elif button_state == "Choose Algorithm":
                if a_star_button.is_clicked(event):
                    path, elapsed_time = pancake.a_star()
                    clear_screen(screen)
                    draw_pancake_secuence(screen, path)
                    state = solved
                    draw_performance(screen, elapsed_time, state)
                    pygame.time.delay(2000)
                    button_state = "Scramble"

                elif mm_button.is_clicked(event):
                    path, elapsed_time = pancake.mm_search("start", "goal")
                    clear_screen(screen)
                    draw_pancake_secuence(screen, path)
                    state = solved
                    draw_performance(screen, elapsed_time, state)
                    pygame.time.delay(2000)
                    button_state = "Scramble"

                elif vs_button.is_clicked(event):
                    pancake2 = Pancake()
                    pancake2 = pancake

                    path1, elapsed_time1 = pancake.a_star()
                    clear_screen(screen)
                    draw_pancake_secuence(screen, path1)
                    draw_performance(screen, elapsed_time1, state)
                    pygame.time.delay(2000)

                    path2, elapsed_time2 = pancake2.mm_search("start", "goal")
                    clear_screen(screen)
                    draw_pancake_secuence(screen, path2)
                    state = solved
                    draw_performance(screen, elapsed_time2, state)
                    pygame.time.delay(2000)              

                    clear_screen(screen)
                    draw_vs(screen, elapsed_time1, elapsed_time2, path1, path2)
                    state = solved
                    button_state = "Scramble"
                    scramble_button.set_text("Scramble")
                    scramble_button.set_color(DARK_ORANGE)
                    

        # Fill the screen with white color
        screen.fill(WHITE)

        # Draw the initial state of the puzzle
        draw_pancake(screen, state)

        if button_state == "Scramble":
            scramble_button.draw(screen)
        elif button_state == "Choose Algorithm":
            a_star_button.draw(screen)
            mm_button.draw(screen)
            vs_button.draw(screen)

        # Update the display
        pygame.display.flip()

        # Limit the frame rate
        clock.tick(fps)

    # Quit Pygame
    pygame.quit()

# Implementation

In [103]:
run_Pancake()

In [104]:
run_8Puzzle()